In [1]:
import time  # 时间库

import numpy as np  # numpy库
import pandas as pd  # pandas库
import pymysql  # mysql连接库

from pyecharts.charts import Bar3D # 3D柱形图

In [2]:
sheet_names = ['2015','2016','2017','2018','会员等级']
sheet_datas = [pd.read_excel('data/sales.xlsx',sheet_name=i) for i in sheet_names]

In [3]:
list(zip(sheet_names,sheet_datas))

[('2015',
                会员ID         订单号       提交日期    订单金额
  0      15278002468  3000304681 2015-01-01   499.0
  1      39236378972  3000305791 2015-01-01  2588.0
  2      38722039578  3000641787 2015-01-01   498.0
  3      11049640063  3000798913 2015-01-01  1572.0
  4      35038752292  3000821546 2015-01-01    10.1
  ...            ...         ...        ...     ...
  30769  39368100847  4281994827 2015-12-31   828.0
  30770       409757  4282010457 2015-12-31   199.0
  30771  38380526114  4282017675 2015-12-31   208.0
  30772     28074988  4282019440 2015-12-31    89.0
  30773  39460363230  4282025309 2015-12-31   719.0
  
  [30774 rows x 4 columns]),
 ('2016',
                会员ID         订单号       提交日期     订单金额
  0      39288120141  4282025766 2016-01-01    76.00
  1      39293812118  4282037929 2016-01-01  7599.00
  2      27596340905  4282038740 2016-01-01   802.00
  3      15111475509  4282043819 2016-01-01    65.00
  4      38896594001  4282051044 2016-01-01    95.00
  ... 

In [4]:
for each_name,each_data in zip(sheet_names,sheet_datas):
    print('[data summary for ============={}===============]'.format(each_name))
    print('Overview:','\n',each_data.head(4))# 展示数据前4条
    print('DESC:','\n',each_data.describe())# 数据描述性信息
    print('NA records',each_data.isnull().any(axis=1).sum()) # 缺失值记录数
    print('Dtypes',each_data.dtypes) # 数据类型

[data summary for =============2015===============]
Overview: 
           会员ID         订单号       提交日期    订单金额
0  15278002468  3000304681 2015-01-01   499.0
1  39236378972  3000305791 2015-01-01  2588.0
2  38722039578  3000641787 2015-01-01   498.0
3  11049640063  3000798913 2015-01-01  1572.0
DESC: 
                会员ID           订单号           订单金额
count  3.077400e+04  3.077400e+04   30774.000000
mean   2.918779e+10  4.020414e+09     960.991161
std    1.385333e+10  2.630510e+08    2068.107231
min    2.670000e+02  3.000305e+09       0.500000
25%    1.944122e+10  3.885510e+09      59.000000
50%    3.746545e+10  4.117491e+09     139.000000
75%    3.923593e+10  4.234882e+09     899.000000
max    3.954613e+10  4.282025e+09  111750.000000
NA records 0
Dtypes 会员ID             int64
订单号              int64
提交日期    datetime64[ns]
订单金额           float64
dtype: object
[data summary for =============2016===============]
Overview: 
           会员ID         订单号       提交日期    订单金额
0  39288120141  42820

In [5]:
# 去除缺失值和异常值
for ind,each_data in enumerate(sheet_datas[:-1]):
    sheet_datas[ind] = each_data.dropna()# 丢弃缺失值记录
    sheet_datas[ind] = each_data[each_data['订单金额'] > 1]# 丢弃订单金额<=1的记录
    sheet_datas[ind]['max_year_date'] = each_data['提交日期'].max() # 增加一列最大日期值

<ipython-input-5-33258259e037>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sheet_datas[ind]['max_year_date'] = each_data['提交日期'].max() # 增加一列最大日期值
<ipython-input-5-33258259e037>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sheet_datas[ind]['max_year_date'] = each_data['提交日期'].max() # 增加一列最大日期值
<ipython-input-5-33258259e037>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [6]:
# 汇总所有数据
data_merge = pd.concat(sheet_datas[:-1],axis=0)
# 获取各自年份数据
data_merge['date_interval'] = data_merge['max_year_date']-data_merge['提交日期']
data_merge['year'] = data_merge['提交日期'].dt.year
# 转换日期间隔为数字
data_merge['date_interval'] = data_merge['date_interval'].apply(lambda x: x.days)
data_merge.head()

会员ID         订单号       提交日期    订单金额 max_year_date  date_interval  \
0  15278002468  3000304681 2015-01-01   499.0    2015-12-31            364   
1  39236378972  3000305791 2015-01-01  2588.0    2015-12-31            364   
2  38722039578  3000641787 2015-01-01   498.0    2015-12-31            364   
3  11049640063  3000798913 2015-01-01  1572.0    2015-12-31            364   
4  35038752292  3000821546 2015-01-01    10.1    2015-12-31            364   

   year  
0  2015  
1  2015  
2  2015  
3  2015  
4  2015

In [7]:
# 按会员ID做汇总
rfm_gb = data_merge.groupby(['year','会员ID'],as_index=False).agg(
    {'date_interval': 'min',  # 计算最近一次订单时间
     '提交日期': 'count', # 计算订单频率
     '订单金额': 'sum'})  # 计算订单总金额
# 重命名列名
rfm_gb.columns =  ['year','会员ID','r','f','m']
rfm_gb.head()

year  会员ID    r  f       m
0  2015   267  197  2   105.0
1  2015   282  251  1    29.7
2  2015   283  340  1  5398.0
3  2015   343  300  1   118.0
4  2015   525   37  3   213.0

In [8]:
# 查看数据分布
desc_pd = rfm_gb.iloc[:,2:].describe().T
desc_pd

count         mean          std  min   25%    50%     75%       max
r  148591.0   165.524043   101.988472  0.0  79.0  156.0   255.0     365.0
f  148591.0     1.365002     2.626953  1.0   1.0    1.0     1.0     130.0
m  148591.0  1323.741329  3753.906883  1.5  69.0  189.0  1199.0  206251.8

In [9]:
# 定义区间边界
r_bins = [-1,79,255,365] # 注意起始边界小于最小值
f_bins = [0,2,5,130] # 通过和业务部门沟通得到
m_bins = [0,69,1199,206252]

In [10]:
# RFM分箱得分
rfm_gb['r_score'] = pd.cut(rfm_gb['r'], r_bins, labels=[i for i in range(len(r_bins)-1,0,-1)])  # 计算R得分
rfm_gb['f_score'] = pd.cut(rfm_gb['f'], f_bins, labels=[i+1 for i in range(len(f_bins)-1)])  # 计算F得分
rfm_gb['m_score'] = pd.cut(rfm_gb['m'], m_bins, labels=[i+1 for i in range(len(m_bins)-1)])  # 计算M得分

In [15]:
rfm_gb

year         会员ID    r  f       m r_score f_score m_score rfm_group
0       2015          267  197  2   105.0       2       1       2       212
1       2015          282  251  1    29.7       2       1       1       211
2       2015          283  340  1  5398.0       1       1       3       113
3       2015          343  300  1   118.0       1       1       2       112
4       2015          525   37  3   213.0       3       2       2       322
...      ...          ...  ... ..     ...     ...     ...     ...       ...
148586  2018  39538034299  272  1    49.0       1       1       1       111
148587  2018  39538034662  189  1  3558.0       2       1       3       213
148588  2018  39538035729  179  1  3699.0       2       1       3       213
148589  2018  39545237824  275  1    49.0       1       1       1       111
148590  2018  39546136285  163  1    19.9       2       1       1       211

[148591 rows x 9 columns]

In [16]:
#计算RFM组合
rfm_gb['r_score'] = rfm_gb['r_score'].astype(np.str)
rfm_gb['f_score'] = rfm_gb['f_score'].astype(np.str)
rfm_gb['m_score'] = rfm_gb['m_score'].astype(np.str)
rfm_gb['rfm_group'] = rfm_gb['r_score'].str.cat(rfm_gb['f_score']).str.cat(
    rfm_gb['m_score'])

<ipython-input-16-6f8e1b242d67>:2: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rfm_gb['r_score'] = rfm_gb['r_score'].astype(np.str)
<ipython-input-16-6f8e1b242d67>:3: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rfm_gb['f_score'] = rfm_gb['f_score'].astype(np.str)
<ipython-input-16-6f8e1b242d67>:4: DeprecationWarning: `np.str` is a deprecated alias for 

In [24]:
rfm_gb

year         会员ID    r  f       m r_score f_score m_score rfm_group
0       2015          267  197  2   105.0       2       1       2       212
1       2015          282  251  1    29.7       2       1       1       211
2       2015          283  340  1  5398.0       1       1       3       113
3       2015          343  300  1   118.0       1       1       2       112
4       2015          525   37  3   213.0       3       2       2       322
...      ...          ...  ... ..     ...     ...     ...     ...       ...
148586  2018  39538034299  272  1    49.0       1       1       1       111
148587  2018  39538034662  189  1  3558.0       2       1       3       213
148588  2018  39538035729  179  1  3699.0       2       1       3       213
148589  2018  39545237824  275  1    49.0       1       1       1       111
148590  2018  39546136285  163  1    19.9       2       1       1       211

[148591 rows x 9 columns]

In [15]:
rfm_gb.to_excel('sales_rfm_score1.xlsx')  # 保存数据为Excel

In [16]:
# 数据库信息
config = {'host': '192.168.88.161',
          'user': 'root',  # 用户名
          'password': '123456',  # 密码
          'port': 3306,  # 端口，默认为3306
          #'database': 'test',  # 数据库名称
          'charset': 'utf8'  # 字符编码
          }
con = pymysql.connect(**config)  # 建立mysql连接
cursor = con.cursor()  # 获得游标

# 创建数据库
create_database_sql = "create database if not exists test"
cursor.execute(create_database_sql)

1

In [17]:
# 数据库信息
config = {'host': '192.168.88.161',
          'user': 'root',  # 用户名
          'password': '123456',  # 密码
          'port': 3306,  # 端口，默认为3306
          'database': 'test',  # 数据库名称
          'charset': 'utf8'  # 字符编码
          }

In [19]:
con = pymysql.connect(**config)  # 建立mysql连接
cursor = con.cursor()  # 获得游标
cursor.execute("show tables")  # 查询表
table_list = [t[0] for t in cursor.fetchall()]  # 读出所有库

In [20]:
table_list

[]

In [21]:
table_name = 'sales_rfm_score'  # 要写库的表名
if not table_name in table_list:  # 如果目标表没有创建
    cursor.execute('''
    CREATE TABLE %s (
    userid               VARCHAR(20),
    r_score               int(2),
    f_score              int(2),
    m_score              int(2),
    rfm_group              VARCHAR(10),
    insert_date              VARCHAR(20)
    )ENGINE=InnoDB DEFAULT CHARSET=utf8
    ''' % table_name)  # 创建新表

In [22]:
# 梳理数据
write_db_data = rfm_gb[['会员ID','r_score','f_score','m_score','rfm_group']] # 主要数据
timestamp = time.strftime('%Y-%m-%d', time.localtime(time.time()))  # 日期

In [24]:
# 写库
for each_value in write_db_data.values:
    insert_sql = "INSERT INTO `%s` VALUES ('%s',%s,%s,%s,'%s','%s')" % \
                 (table_name, each_value[0], each_value[1], each_value[2],
                  each_value[3],each_value[4],
                  timestamp)  # 写库SQL依据
    cursor.execute(insert_sql)  # 执行SQL语句，execute函数里面要用双引号
    con.commit()  # 提交命令
cursor.close()  # 关闭游标
con.close()  # 关闭数据库连接

In [25]:
# 图形数据汇总
display_data = rfm_gb.groupby(['rfm_group','year'],as_index=False)['会员ID'].count()
display_data.columns = ['rfm_group','year','number']
display_data['rfm_group'] = display_data['rfm_group'].astype(np.int32)
display_data.head()

rfm_group  year  number
0        111  2015    2180
1        111  2016    1498
2        111  2017    3169
3        111  2018    2271
4        112  2015    3811

In [26]:
 [d.tolist() for d in display_data.values]

[[111, 2015, 2180],
 [111, 2016, 1498],
 [111, 2017, 3169],
 [111, 2018, 2271],
 [112, 2015, 3811],
 [112, 2016, 2958],
 [112, 2017, 5737],
 [112, 2018, 4340],
 [113, 2015, 1785],
 [113, 2016, 1297],
 [113, 2017, 2506],
 [113, 2018, 1947],
 [121, 2017, 1],
 [121, 2018, 8],
 [122, 2015, 27],
 [122, 2017, 12],
 [122, 2018, 976],
 [123, 2015, 24],
 [123, 2017, 4],
 [123, 2018, 1904],
 [132, 2018, 2],
 [133, 2015, 15],
 [133, 2016, 233],
 [133, 2017, 36],
 [133, 2018, 188],
 [211, 2015, 3532],
 [211, 2016, 4678],
 [211, 2017, 3509],
 [211, 2018, 7304],
 [212, 2015, 6576],
 [212, 2016, 8983],
 [212, 2017, 6983],
 [212, 2018, 14297],
 [213, 2015, 2962],
 [213, 2016, 4036],
 [213, 2017, 2983],
 [213, 2018, 6389],
 [221, 2015, 2],
 [221, 2016, 6],
 [221, 2017, 3],
 [221, 2018, 13],
 [222, 2015, 43],
 [222, 2016, 69],
 [222, 2017, 46],
 [222, 2018, 216],
 [223, 2015, 37],
 [223, 2016, 42],
 [223, 2017, 38],
 [223, 2018, 253],
 [232, 2016, 4],
 [232, 2018, 9],
 [233, 2015, 20],
 [233, 2016, 50],

In [27]:
# 显示图形
from pyecharts.charts import Bar3D
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
range_color = ['#313695', '#4575b4', '#74add1', '#abd9e9', '#e0f3f8', '#ffffbf',
               '#fee090', '#fdae61', '#f46d43', '#d73027', '#a50026']
range_max = int(display_data['number'].max())
c = (
    Bar3D()#设置了一个3D柱形图对象
        .add(
        "",#标题
        [d.tolist() for d in display_data.values],#数据
        xaxis3d_opts=opts.Axis3DOpts( type_="category",name='分组名称'),#x轴数据类型，名称
        yaxis3d_opts=opts.Axis3DOpts( type_="category",name='年份'),#y轴数据类型，名称
        zaxis3d_opts=opts.Axis3DOpts(type_="value",name='会员数量'),#z轴数据类型，名称
    )
        .set_global_opts(#设置颜色，及不同取值对应的颜色
        visualmap_opts=opts.VisualMapOpts(max_=range_max,range_color=range_color),
        title_opts=opts.TitleOpts(title="RFM分组结果"),#设置标题
    )
)
c.render() #在html中显示

'/root/render.html'